In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
X_train = mnist.train.images
Y_train = mnist.train.labels
X_test = mnist.test.images
Y_test = mnist.test.labels

In [4]:
dimX = X_train.shape[1]
dimY = Y_train.shape[1]
nTrain = X_train.shape[0]
nTest = X_test.shape[0]

In [5]:
nLayer0 = dimX
nLayer1 = 256
nLayer2 = 256
nLayer3 = dimY
sigma_init = 0.1

In [6]:
W = {
    'W1': tf.Variable(tf.random_normal([nLayer0, nLayer1], stddev = sigma_init)),
    'W2': tf.Variable(tf.random_normal([nLayer1, nLayer2], stddev = sigma_init)),
    'W3': tf.Variable(tf.random_normal([nLayer2, nLayer3], stddev = sigma_init))
}

b = {
    'b1': tf.Variable(tf.random_normal([nLayer1])),
    'b2': tf.Variable(tf.random_normal([nLayer2])),
    'b3': tf.Variable(tf.random_normal([nLayer3]))
}

In [7]:
def model_myNN(_X, _W, _b):
    Layer1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _W['W1']), _b['b1']))
    Layer2 = tf.nn.sigmoid(tf.add(tf.matmul(Layer1, _W['W2']), _b['b2']))
    Layer3 = tf.add(tf.matmul(Layer2, _W['W3']), _b['b3'])
    return Layer3    

In [8]:
X = tf.placeholder(tf.float32, [None, dimX], name="input")
Y = tf.placeholder(tf.float32, [None, dimY], name="output")

In [9]:
Y_pred = model_myNN(X, W, b)

In [12]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y_pred, labels=Y))

In [18]:
learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

training_epochs = 30
display_epoch = 5
batch_size = 100

In [19]:
correct_prediction = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [20]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        nBatch = int(nTrain/batch_size)
        myIdx = np.random.permutation(nTrain)
        
        for ii in range(nBatch):
            X_batch = X_train[myIdx[ii * batch_size: (ii+1)*batch_size], :]
            Y_batch = Y_train[myIdx[ii * batch_size: (ii+1)*batch_size], :]
            sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})
            
        if (epoch+1) % display_epoch == 0:
            loss_temp = sess.run(loss, feed_dict={X: X_train, Y: Y_train})
            accuracy_temp = accuracy.eval({X: X_train, Y: Y_train})
            print("epoch{}".format(epoch+1))
            print("[Loss / Training Accuracy] {:05.4f} / {:05.4f}".format(loss_temp, accuracy_temp))
            print(" ")
            
    print("[Test Accuracy]", accuracy.eval({X: X_test, Y: Y_test}))

epoch5
[Loss / Training Accuracy] 0.0786 / 0.9776
 
epoch10
[Loss / Training Accuracy] 0.0245 / 0.9937
 
epoch15
[Loss / Training Accuracy] 0.0089 / 0.9981
 
epoch20
[Loss / Training Accuracy] 0.0025 / 0.9997
 
epoch25
[Loss / Training Accuracy] 0.0004 / 1.0000
 
epoch30
[Loss / Training Accuracy] 0.0004 / 1.0000
 
[Test Accuracy] 0.9807
